In [50]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/av-jobathon-sep2021/sample.csv
/kaggle/input/av-jobathon-sep2021/train.csv
/kaggle/input/av-jobathon-sep2021/test.csv


## Reading the datasets

In [51]:
df = pd.read_csv("/kaggle/input/av-jobathon-sep2021/train.csv")
df

,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount,#Order,Sales
0,T1000001,1,S1,L3,R1,2018-01-01,1,Yes,9,7011.84
1,T1000002,253,S4,L2,R1,2018-01-01,1,Yes,60,51789.12
2,T1000003,252,S3,L2,R1,2018-01-01,1,Yes,42,36868.20
3,T1000004,251,S2,L3,R1,2018-01-01,1,Yes,23,19715.16
4,T1000005,250,S2,L3,R4,2018-01-01,1,Yes,62,45614.52
...,...,...,...,...,...,...,...,...,...,...
188335,T1188336,149,S2,L3,R2,2019-05-31,1,Yes,51,37272.00
188336,T1188337,153,S4,L2,R1,2019-05-31,1,No,90,54572.64
188337,T1188338,154,S1,L3,R2,2019-05-31,1,No,56,31624.56
188338,T1188339,155,S3,L1,R2,2019-05-31,1,Yes,70,49162.41


## Exploratory Data Analysis

In [52]:
import pandas_profiling as pp

In [53]:
profile = pp.ProfileReport(df)

In [54]:
#profile

### Checking for the missing values in the data

In [55]:
df.isnull().sum()

ID               0
Store_id         0
Store_Type       0
Location_Type    0
Region_Code      0
Date             0
Holiday          0
Discount         0
#Order           0
Sales            0
dtype: int64

There is no missing values in the training data

Since the transcation id is unique we don't need that in our analysis

In [56]:
df.drop(["ID"],axis=1,inplace=True)

In [57]:
df.head()

,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount,#Order,Sales
0,1,S1,L3,R1,2018-01-01,1,Yes,9,7011.84
1,253,S4,L2,R1,2018-01-01,1,Yes,60,51789.12
2,252,S3,L2,R1,2018-01-01,1,Yes,42,36868.20
3,251,S2,L3,R1,2018-01-01,1,Yes,23,19715.16
4,250,S2,L3,R4,2018-01-01,1,Yes,62,45614.52


## Pre-processing 

There are 5 categorical columns and one numerical columns in the data.
For each of the categorical columns we need to convert it into numerical data, so we perform one hot encoding for the categorical features.

In [58]:
encoded_data = pd.get_dummies(df,columns=["Store_Type","Location_Type","Region_Code","Discount"])
encoded_data

,Store_id,Date,Holiday,#Order,Sales,Store_Type_S1,Store_Type_S2,Store_Type_S3,Store_Type_S4,Location_Type_L1,Location_Type_L2,Location_Type_L3,Location_Type_L4,Location_Type_L5,Region_Code_R1,Region_Code_R2,Region_Code_R3,Region_Code_R4,Discount_No,Discount_Yes
0,1,2018-01-01,1,9,7011.84,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1
1,253,2018-01-01,1,60,51789.12,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1
2,252,2018-01-01,1,42,36868.20,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1
3,251,2018-01-01,1,23,19715.16,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1
4,250,2018-01-01,1,62,45614.52,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188335,149,2019-05-31,1,51,37272.00,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1
188336,153,2019-05-31,1,90,54572.64,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0
188337,154,2019-05-31,1,56,31624.56,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0
188338,155,2019-05-31,1,70,49162.41,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1


#### Importing the test data and performing the transformation on it

In [59]:
test = pd.read_csv("/kaggle/input/av-jobathon-sep2021/test.csv")
test

,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount
0,T1188341,171,S4,L2,R3,2019-06-01,0,No
1,T1188342,172,S1,L1,R1,2019-06-01,0,No
2,T1188343,173,S4,L2,R1,2019-06-01,0,No
3,T1188344,174,S1,L1,R4,2019-06-01,0,No
4,T1188345,170,S1,L1,R2,2019-06-01,0,No
...,...,...,...,...,...,...,...,...
22260,T1210601,186,S2,L5,R2,2019-07-31,0,No
22261,T1210602,11,S4,L2,R1,2019-07-31,0,No
22262,T1210603,185,S1,L1,R3,2019-07-31,0,Yes
22263,T1210604,69,S1,L1,R4,2019-07-31,0,No


In [60]:
encoded_test_data = pd.get_dummies(test,columns=["Store_Type","Location_Type","Region_Code","Discount"])
encoded_test_data

,ID,Store_id,Date,Holiday,Store_Type_S1,Store_Type_S2,Store_Type_S3,Store_Type_S4,Location_Type_L1,Location_Type_L2,Location_Type_L3,Location_Type_L4,Location_Type_L5,Region_Code_R1,Region_Code_R2,Region_Code_R3,Region_Code_R4,Discount_No,Discount_Yes
0,T1188341,171,2019-06-01,0,0,0,0,1,0,1,0,0,0,0,0,1,0,1,0
1,T1188342,172,2019-06-01,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0
2,T1188343,173,2019-06-01,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0
3,T1188344,174,2019-06-01,0,1,0,0,0,1,0,0,0,0,0,0,0,1,1,0
4,T1188345,170,2019-06-01,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22260,T1210601,186,2019-07-31,0,0,1,0,0,0,0,0,0,1,0,1,0,0,1,0
22261,T1210602,11,2019-07-31,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0
22262,T1210603,185,2019-07-31,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1
22263,T1210604,69,2019-07-31,0,1,0,0,0,1,0,0,0,0,0,0,0,1,1,0


## Feature Engineering
### Extracting the Days of the week using date

In [61]:
#training_set
encoded_data['Date'] = pd.to_datetime(encoded_data['Date'])
encoded_data['Day']=encoded_data['Date'].dt.day_name()
encoded_data

,Store_id,Date,Holiday,#Order,Sales,Store_Type_S1,Store_Type_S2,Store_Type_S3,Store_Type_S4,Location_Type_L1,...,Location_Type_L3,Location_Type_L4,Location_Type_L5,Region_Code_R1,Region_Code_R2,Region_Code_R3,Region_Code_R4,Discount_No,Discount_Yes,Day
0,1,2018-01-01,1,9,7011.84,1,0,0,0,0,...,1,0,0,1,0,0,0,0,1,Monday
1,253,2018-01-01,1,60,51789.12,0,0,0,1,0,...,0,0,0,1,0,0,0,0,1,Monday
2,252,2018-01-01,1,42,36868.20,0,0,1,0,0,...,0,0,0,1,0,0,0,0,1,Monday
3,251,2018-01-01,1,23,19715.16,0,1,0,0,0,...,1,0,0,1,0,0,0,0,1,Monday
4,250,2018-01-01,1,62,45614.52,0,1,0,0,0,...,1,0,0,0,0,0,1,0,1,Monday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188335,149,2019-05-31,1,51,37272.00,0,1,0,0,0,...,1,0,0,0,1,0,0,0,1,Friday
188336,153,2019-05-31,1,90,54572.64,0,0,0,1,0,...,0,0,0,1,0,0,0,1,0,Friday
188337,154,2019-05-31,1,56,31624.56,1,0,0,0,0,...,1,0,0,0,1,0,0,1,0,Friday
188338,155,2019-05-31,1,70,49162.41,0,0,1,0,1,...,0,0,0,0,1,0,0,0,1,Friday


In [62]:
#training_set
encoded_test_data['Date'] = pd.to_datetime(encoded_test_data['Date'])
encoded_test_data['Day']=encoded_test_data['Date'].dt.day_name()
encoded_test_data

,ID,Store_id,Date,Holiday,Store_Type_S1,Store_Type_S2,Store_Type_S3,Store_Type_S4,Location_Type_L1,Location_Type_L2,Location_Type_L3,Location_Type_L4,Location_Type_L5,Region_Code_R1,Region_Code_R2,Region_Code_R3,Region_Code_R4,Discount_No,Discount_Yes,Day
0,T1188341,171,2019-06-01,0,0,0,0,1,0,1,0,0,0,0,0,1,0,1,0,Saturday
1,T1188342,172,2019-06-01,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,Saturday
2,T1188343,173,2019-06-01,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,Saturday
3,T1188344,174,2019-06-01,0,1,0,0,0,1,0,0,0,0,0,0,0,1,1,0,Saturday
4,T1188345,170,2019-06-01,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,Saturday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22260,T1210601,186,2019-07-31,0,0,1,0,0,0,0,0,0,1,0,1,0,0,1,0,Wednesday
22261,T1210602,11,2019-07-31,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,Wednesday
22262,T1210603,185,2019-07-31,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,Wednesday
22263,T1210604,69,2019-07-31,0,1,0,0,0,1,0,0,0,0,0,0,0,1,1,0,Wednesday


### Encoding the Day Column

In [63]:
#training_data
encoded_data = pd.get_dummies(encoded_data,columns=["Day"]) 
encoded_data

,Store_id,Date,Holiday,#Order,Sales,Store_Type_S1,Store_Type_S2,Store_Type_S3,Store_Type_S4,Location_Type_L1,...,Region_Code_R4,Discount_No,Discount_Yes,Day_Friday,Day_Monday,Day_Saturday,Day_Sunday,Day_Thursday,Day_Tuesday,Day_Wednesday
0,1,2018-01-01,1,9,7011.84,1,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
1,253,2018-01-01,1,60,51789.12,0,0,0,1,0,...,0,0,1,0,1,0,0,0,0,0
2,252,2018-01-01,1,42,36868.20,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
3,251,2018-01-01,1,23,19715.16,0,1,0,0,0,...,0,0,1,0,1,0,0,0,0,0
4,250,2018-01-01,1,62,45614.52,0,1,0,0,0,...,1,0,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188335,149,2019-05-31,1,51,37272.00,0,1,0,0,0,...,0,0,1,1,0,0,0,0,0,0
188336,153,2019-05-31,1,90,54572.64,0,0,0,1,0,...,0,1,0,1,0,0,0,0,0,0
188337,154,2019-05-31,1,56,31624.56,1,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0
188338,155,2019-05-31,1,70,49162.41,0,0,1,0,1,...,0,0,1,1,0,0,0,0,0,0


In [64]:
#test_data
encoded_test_data = pd.get_dummies(encoded_test_data,columns=["Day"])

In [65]:
encoded_test_data

,ID,Store_id,Date,Holiday,Store_Type_S1,Store_Type_S2,Store_Type_S3,Store_Type_S4,Location_Type_L1,Location_Type_L2,...,Region_Code_R4,Discount_No,Discount_Yes,Day_Friday,Day_Monday,Day_Saturday,Day_Sunday,Day_Thursday,Day_Tuesday,Day_Wednesday
0,T1188341,171,2019-06-01,0,0,0,0,1,0,1,...,0,1,0,0,0,1,0,0,0,0
1,T1188342,172,2019-06-01,0,1,0,0,0,1,0,...,0,1,0,0,0,1,0,0,0,0
2,T1188343,173,2019-06-01,0,0,0,0,1,0,1,...,0,1,0,0,0,1,0,0,0,0
3,T1188344,174,2019-06-01,0,1,0,0,0,1,0,...,1,1,0,0,0,1,0,0,0,0
4,T1188345,170,2019-06-01,0,1,0,0,0,1,0,...,0,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22260,T1210601,186,2019-07-31,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
22261,T1210602,11,2019-07-31,0,0,0,0,1,0,1,...,0,1,0,0,0,0,0,0,0,1
22262,T1210603,185,2019-07-31,0,1,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,1
22263,T1210604,69,2019-07-31,0,1,0,0,0,1,0,...,1,1,0,0,0,0,0,0,0,1


### Extracting the month from date column

In [66]:
#training_set
encoded_data["month"] = pd.DatetimeIndex(encoded_data["Date"]).month
encoded_data

,Store_id,Date,Holiday,#Order,Sales,Store_Type_S1,Store_Type_S2,Store_Type_S3,Store_Type_S4,Location_Type_L1,...,Discount_No,Discount_Yes,Day_Friday,Day_Monday,Day_Saturday,Day_Sunday,Day_Thursday,Day_Tuesday,Day_Wednesday,month
0,1,2018-01-01,1,9,7011.84,1,0,0,0,0,...,0,1,0,1,0,0,0,0,0,1
1,253,2018-01-01,1,60,51789.12,0,0,0,1,0,...,0,1,0,1,0,0,0,0,0,1
2,252,2018-01-01,1,42,36868.20,0,0,1,0,0,...,0,1,0,1,0,0,0,0,0,1
3,251,2018-01-01,1,23,19715.16,0,1,0,0,0,...,0,1,0,1,0,0,0,0,0,1
4,250,2018-01-01,1,62,45614.52,0,1,0,0,0,...,0,1,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188335,149,2019-05-31,1,51,37272.00,0,1,0,0,0,...,0,1,1,0,0,0,0,0,0,5
188336,153,2019-05-31,1,90,54572.64,0,0,0,1,0,...,1,0,1,0,0,0,0,0,0,5
188337,154,2019-05-31,1,56,31624.56,1,0,0,0,0,...,1,0,1,0,0,0,0,0,0,5
188338,155,2019-05-31,1,70,49162.41,0,0,1,0,1,...,0,1,1,0,0,0,0,0,0,5


In [67]:
#test_set
encoded_test_data["month"] = pd.DatetimeIndex(encoded_test_data["Date"]).month
encoded_data

,Store_id,Date,Holiday,#Order,Sales,Store_Type_S1,Store_Type_S2,Store_Type_S3,Store_Type_S4,Location_Type_L1,...,Discount_No,Discount_Yes,Day_Friday,Day_Monday,Day_Saturday,Day_Sunday,Day_Thursday,Day_Tuesday,Day_Wednesday,month
0,1,2018-01-01,1,9,7011.84,1,0,0,0,0,...,0,1,0,1,0,0,0,0,0,1
1,253,2018-01-01,1,60,51789.12,0,0,0,1,0,...,0,1,0,1,0,0,0,0,0,1
2,252,2018-01-01,1,42,36868.20,0,0,1,0,0,...,0,1,0,1,0,0,0,0,0,1
3,251,2018-01-01,1,23,19715.16,0,1,0,0,0,...,0,1,0,1,0,0,0,0,0,1
4,250,2018-01-01,1,62,45614.52,0,1,0,0,0,...,0,1,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188335,149,2019-05-31,1,51,37272.00,0,1,0,0,0,...,0,1,1,0,0,0,0,0,0,5
188336,153,2019-05-31,1,90,54572.64,0,0,0,1,0,...,1,0,1,0,0,0,0,0,0,5
188337,154,2019-05-31,1,56,31624.56,1,0,0,0,0,...,1,0,1,0,0,0,0,0,0,5
188338,155,2019-05-31,1,70,49162.41,0,0,1,0,1,...,0,1,1,0,0,0,0,0,0,5


### Performing encoding on months

In [68]:
#training_data
encoded_data = pd.get_dummies(encoded_data,columns=["month"]) 
encoded_data

,Store_id,Date,Holiday,#Order,Sales,Store_Type_S1,Store_Type_S2,Store_Type_S3,Store_Type_S4,Location_Type_L1,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,1,2018-01-01,1,9,7011.84,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,253,2018-01-01,1,60,51789.12,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,252,2018-01-01,1,42,36868.20,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,251,2018-01-01,1,23,19715.16,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,250,2018-01-01,1,62,45614.52,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188335,149,2019-05-31,1,51,37272.00,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
188336,153,2019-05-31,1,90,54572.64,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
188337,154,2019-05-31,1,56,31624.56,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
188338,155,2019-05-31,1,70,49162.41,0,0,1,0,1,...,0,0,1,0,0,0,0,0,0,0


In [69]:
#test_data
encoded_test_data = pd.get_dummies(encoded_test_data,columns=["month"]) 
encoded_test_data

,ID,Store_id,Date,Holiday,Store_Type_S1,Store_Type_S2,Store_Type_S3,Store_Type_S4,Location_Type_L1,Location_Type_L2,...,Discount_Yes,Day_Friday,Day_Monday,Day_Saturday,Day_Sunday,Day_Thursday,Day_Tuesday,Day_Wednesday,month_6,month_7
0,T1188341,171,2019-06-01,0,0,0,0,1,0,1,...,0,0,0,1,0,0,0,0,1,0
1,T1188342,172,2019-06-01,0,1,0,0,0,1,0,...,0,0,0,1,0,0,0,0,1,0
2,T1188343,173,2019-06-01,0,0,0,0,1,0,1,...,0,0,0,1,0,0,0,0,1,0
3,T1188344,174,2019-06-01,0,1,0,0,0,1,0,...,0,0,0,1,0,0,0,0,1,0
4,T1188345,170,2019-06-01,0,1,0,0,0,1,0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22260,T1210601,186,2019-07-31,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
22261,T1210602,11,2019-07-31,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,1,0,1
22262,T1210603,185,2019-07-31,0,1,0,0,0,1,0,...,1,0,0,0,0,0,0,1,0,1
22263,T1210604,69,2019-07-31,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,1


In [70]:
encoded_test_data

,ID,Store_id,Date,Holiday,Store_Type_S1,Store_Type_S2,Store_Type_S3,Store_Type_S4,Location_Type_L1,Location_Type_L2,...,Discount_Yes,Day_Friday,Day_Monday,Day_Saturday,Day_Sunday,Day_Thursday,Day_Tuesday,Day_Wednesday,month_6,month_7
0,T1188341,171,2019-06-01,0,0,0,0,1,0,1,...,0,0,0,1,0,0,0,0,1,0
1,T1188342,172,2019-06-01,0,1,0,0,0,1,0,...,0,0,0,1,0,0,0,0,1,0
2,T1188343,173,2019-06-01,0,0,0,0,1,0,1,...,0,0,0,1,0,0,0,0,1,0
3,T1188344,174,2019-06-01,0,1,0,0,0,1,0,...,0,0,0,1,0,0,0,0,1,0
4,T1188345,170,2019-06-01,0,1,0,0,0,1,0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22260,T1210601,186,2019-07-31,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
22261,T1210602,11,2019-07-31,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,1,0,1
22262,T1210603,185,2019-07-31,0,1,0,0,0,1,0,...,1,0,0,0,0,0,0,1,0,1
22263,T1210604,69,2019-07-31,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,1


### Checking wheather all columns are same in training and test data

In [71]:
encoded_data.columns

Index(['Store_id', 'Date', 'Holiday', '#Order', 'Sales', 'Store_Type_S1',
       'Store_Type_S2', 'Store_Type_S3', 'Store_Type_S4', 'Location_Type_L1',
       'Location_Type_L2', 'Location_Type_L3', 'Location_Type_L4',
       'Location_Type_L5', 'Region_Code_R1', 'Region_Code_R2',
       'Region_Code_R3', 'Region_Code_R4', 'Discount_No', 'Discount_Yes',
       'Day_Friday', 'Day_Monday', 'Day_Saturday', 'Day_Sunday',
       'Day_Thursday', 'Day_Tuesday', 'Day_Wednesday', 'month_1', 'month_2',
       'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_8',
       'month_9', 'month_10', 'month_11', 'month_12'],
      dtype='object')

In [72]:
encoded_test_data.columns

Index(['ID', 'Store_id', 'Date', 'Holiday', 'Store_Type_S1', 'Store_Type_S2',
       'Store_Type_S3', 'Store_Type_S4', 'Location_Type_L1',
       'Location_Type_L2', 'Location_Type_L3', 'Location_Type_L4',
       'Location_Type_L5', 'Region_Code_R1', 'Region_Code_R2',
       'Region_Code_R3', 'Region_Code_R4', 'Discount_No', 'Discount_Yes',
       'Day_Friday', 'Day_Monday', 'Day_Saturday', 'Day_Sunday',
       'Day_Thursday', 'Day_Tuesday', 'Day_Wednesday', 'month_6', 'month_7'],
      dtype='object')

### There is and extra column in test data Id and #order 

In [73]:
encoded_test_data.drop(["ID"],axis=1,inplace=True)

In [74]:
encoded_data.drop(["#Order"],axis=1,inplace=True)

In [75]:
encoded_data.columns

Index(['Store_id', 'Date', 'Holiday', 'Sales', 'Store_Type_S1',
       'Store_Type_S2', 'Store_Type_S3', 'Store_Type_S4', 'Location_Type_L1',
       'Location_Type_L2', 'Location_Type_L3', 'Location_Type_L4',
       'Location_Type_L5', 'Region_Code_R1', 'Region_Code_R2',
       'Region_Code_R3', 'Region_Code_R4', 'Discount_No', 'Discount_Yes',
       'Day_Friday', 'Day_Monday', 'Day_Saturday', 'Day_Sunday',
       'Day_Thursday', 'Day_Tuesday', 'Day_Wednesday', 'month_1', 'month_2',
       'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_8',
       'month_9', 'month_10', 'month_11', 'month_12'],
      dtype='object')

In [76]:
encoded_test_data.columns

Index(['Store_id', 'Date', 'Holiday', 'Store_Type_S1', 'Store_Type_S2',
       'Store_Type_S3', 'Store_Type_S4', 'Location_Type_L1',
       'Location_Type_L2', 'Location_Type_L3', 'Location_Type_L4',
       'Location_Type_L5', 'Region_Code_R1', 'Region_Code_R2',
       'Region_Code_R3', 'Region_Code_R4', 'Discount_No', 'Discount_Yes',
       'Day_Friday', 'Day_Monday', 'Day_Saturday', 'Day_Sunday',
       'Day_Thursday', 'Day_Tuesday', 'Day_Wednesday', 'month_6', 'month_7'],
      dtype='object')

### Viewing the final datasets

In [77]:
encoded_data.head()

,Store_id,Date,Holiday,Sales,Store_Type_S1,Store_Type_S2,Store_Type_S3,Store_Type_S4,Location_Type_L1,Location_Type_L2,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,1,2018-01-01,1,7011.84,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,253,2018-01-01,1,51789.12,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,252,2018-01-01,1,36868.20,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,251,2018-01-01,1,19715.16,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,250,2018-01-01,1,45614.52,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [78]:
encoded_test_data.head()

,Store_id,Date,Holiday,Store_Type_S1,Store_Type_S2,Store_Type_S3,Store_Type_S4,Location_Type_L1,Location_Type_L2,Location_Type_L3,...,Discount_Yes,Day_Friday,Day_Monday,Day_Saturday,Day_Sunday,Day_Thursday,Day_Tuesday,Day_Wednesday,month_6,month_7
0,171,2019-06-01,0,0,0,0,1,0,1,0,...,0,0,0,1,0,0,0,0,1,0
1,172,2019-06-01,0,1,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,1,0
2,173,2019-06-01,0,0,0,0,1,0,1,0,...,0,0,0,1,0,0,0,0,1,0
3,174,2019-06-01,0,1,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,1,0
4,170,2019-06-01,0,1,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,1,0


we don't need the data column in train and test data now, so we drop the columns in both the datasets

In [79]:
encoded_data.drop(["Date"],axis=1,inplace=True)
encoded_test_data.drop(["Date"],axis=1,inplace=True)

In [80]:
encoded_data.head()

,Store_id,Holiday,Sales,Store_Type_S1,Store_Type_S2,Store_Type_S3,Store_Type_S4,Location_Type_L1,Location_Type_L2,Location_Type_L3,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,1,1,7011.84,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,253,1,51789.12,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,252,1,36868.20,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,251,1,19715.16,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,250,1,45614.52,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [81]:
encoded_test_data.head()

,Store_id,Holiday,Store_Type_S1,Store_Type_S2,Store_Type_S3,Store_Type_S4,Location_Type_L1,Location_Type_L2,Location_Type_L3,Location_Type_L4,...,Discount_Yes,Day_Friday,Day_Monday,Day_Saturday,Day_Sunday,Day_Thursday,Day_Tuesday,Day_Wednesday,month_6,month_7
0,171,0,0,0,0,1,0,1,0,0,...,0,0,0,1,0,0,0,0,1,0
1,172,0,1,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,173,0,0,0,0,1,0,1,0,0,...,0,0,0,1,0,0,0,0,1,0
3,174,0,1,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
4,170,0,1,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0


## Preparing the data for training the model 

In [82]:
from sklearn.model_selection import train_test_split

In [83]:
x = encoded_data.drop(["Sales"],axis=1)

In [84]:
y = encoded_data["Sales"]
y

0          7011.84
1         51789.12
2         36868.20
3         19715.16
4         45614.52
            ...   
188335    37272.00
188336    54572.64
188337    31624.56
188338    49162.41
188339    37977.00
Name: Sales, Length: 188340, dtype: float64

In [85]:
#splitting the dataset for training the model
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.3)

### Linear Regression

In [86]:
from sklearn.linear_model import LinearRegression

In [87]:
LR = LinearRegression()

In [88]:
LR.fit(X_train,y_train)

LinearRegression()

In [89]:
y_prediction =  LR.predict(X_test)
y_prediction

array([26436.84375, 30071.21875, 72563.03125, ..., 61418.9375 ,
       17320.6875 , 44881.09375])

In [90]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
# predicting the accuracy score
score=r2_score(y_test,y_prediction)
print("r2 socre is ",score)
print("mean_sqrd_error is==",mean_squared_error(y_test,y_prediction))
print("root_mean_squared error of is==",np.sqrt(mean_squared_error(y_test,y_prediction)))

r2 socre is  0.6025031680599162
mean_sqrd_error is== 135440838.43213806
root_mean_squared error of is== 11637.905242445397


### XGBOOSTER

In [91]:
import xgboost as xg

In [92]:
xgb_r = xg.XGBRegressor(objective ='reg:linear',
                  n_estimators = 10, seed = 123)

In [93]:
xgb_r.fit(X_train, y_train)


[13:24:10] WARNING: ../src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=10, n_jobs=4, num_parallel_tree=1,
             objective='reg:linear', random_state=123, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, seed=123, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [94]:
pred = xgb_r.predict(X_test)

In [95]:
pred

array([28867.113, 32089.89 , 65795.445, ..., 58796.5  , 20442.271,
       35574.94 ], dtype=float32)

In [96]:
y_test

161300    21204.00
161415    30471.00
48977     66324.00
81115     50178.00
93268     44069.85
            ...   
127175    53739.00
136708    78912.00
123959    58518.00
41361     11328.00
48249     34995.00
Name: Sales, Length: 56502, dtype: float64

In [97]:
from sklearn.metrics import mean_squared_error as MSE
rmse = np.sqrt(MSE(y_test, pred))
print("RMSE : % f" %(rmse))

RMSE :  10650.405854


In [107]:
encoded_test_data

,Store_id,Holiday,Store_Type_S1,Store_Type_S2,Store_Type_S3,Store_Type_S4,Location_Type_L1,Location_Type_L2,Location_Type_L3,Location_Type_L4,...,Discount_Yes,Day_Friday,Day_Monday,Day_Saturday,Day_Sunday,Day_Thursday,Day_Tuesday,Day_Wednesday,month_6,month_7
0,171,0,0,0,0,1,0,1,0,0,...,0,0,0,1,0,0,0,0,1,0
1,172,0,1,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,173,0,0,0,0,1,0,1,0,0,...,0,0,0,1,0,0,0,0,1,0
3,174,0,1,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
4,170,0,1,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22260,186,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
22261,11,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,1,0,1
22262,185,0,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,1,0,1
22263,69,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,1


There are few columns missing in the encoded_test_data, month_1 to month_12 excluding month_6 and month_7 is missing

In [119]:
encoded_test_data["month_1"] = encoded_test_data["month_6"]
encoded_test_data["month_2"] = encoded_test_data["month_6"]
encoded_test_data["month_3"] = encoded_test_data["month_6"]
encoded_test_data["month_4"] = encoded_test_data["month_6"]
encoded_test_data["month_5"] = encoded_test_data["month_6"]
encoded_test_data["month_6_new"] = encoded_test_data["month_6"]
encoded_test_data["month_7_new"] = encoded_test_data["month_7"]
encoded_test_data["month_8"] = encoded_test_data["month_6"]
encoded_test_data["month_9"] = encoded_test_data["month_6"]
encoded_test_data["month_10"] = encoded_test_data["month_6"]
encoded_test_data["month_11"] = encoded_test_data["month_6"]
encoded_test_data["month_12"] = encoded_test_data["month_6"]


encoded_test_data

,Store_id,Holiday,Store_Type_S1,Store_Type_S2,Store_Type_S3,Store_Type_S4,Location_Type_L1,Location_Type_L2,Location_Type_L3,Location_Type_L4,...,month_3,month_4,month_5,month_6_new,month_7_new,month_8,month_9,month_10,month_11,month_12
0,171,0,0,0,0,1,0,1,0,0,...,1,1,1,1,0,1,1,1,1,1
1,172,0,1,0,0,0,1,0,0,0,...,1,1,1,1,0,1,1,1,1,1
2,173,0,0,0,0,1,0,1,0,0,...,1,1,1,1,0,1,1,1,1,1
3,174,0,1,0,0,0,1,0,0,0,...,1,1,1,1,0,1,1,1,1,1
4,170,0,1,0,0,0,1,0,0,0,...,1,1,1,1,0,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22260,186,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
22261,11,0,0,0,0,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
22262,185,0,1,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
22263,69,0,1,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [122]:
encoded_test_data.drop(["month_6","month_7"],axis=1,inplace=True)

In [126]:
encoded_test_data.rename(columns={"month_6_new":"month_6","month_7_new":"month_7"},inplace=True)

In [128]:
cols=["month_1","month_2","month_3","month_4","month_5","month_8","month_9","month_10","month_11","month_12"]
cols

['month_1',
 'month_2',
 'month_3',
 'month_4',
 'month_5',
 'month_8',
 'month_9',
 'month_10',
 'month_11',
 'month_12']

In [136]:
for i in cols:
    encoded_test_data[cols]=0

In [137]:
encoded_test_data[cols]

,month_1,month_2,month_3,month_4,month_5,month_8,month_9,month_10,month_11,month_12
0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
22260,0,0,0,0,0,0,0,0,0,0
22261,0,0,0,0,0,0,0,0,0,0
22262,0,0,0,0,0,0,0,0,0,0
22263,0,0,0,0,0,0,0,0,0,0


In [138]:
encoded_test_data

,Store_id,Holiday,Store_Type_S1,Store_Type_S2,Store_Type_S3,Store_Type_S4,Location_Type_L1,Location_Type_L2,Location_Type_L3,Location_Type_L4,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,171,0,0,0,0,1,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
1,172,0,1,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,173,0,0,0,0,1,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
3,174,0,1,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,170,0,1,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22260,186,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
22261,11,0,0,0,0,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
22262,185,0,1,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
22263,69,0,1,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [139]:
result = xgb_r.predict(encoded_test_data)

In [141]:
result

array([67248.875, 40456.45 , 73546.3  , ..., 43249.37 , 33266.293,
       30043.516], dtype=float32)

In [142]:
test

,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount
0,T1188341,171,S4,L2,R3,2019-06-01,0,No
1,T1188342,172,S1,L1,R1,2019-06-01,0,No
2,T1188343,173,S4,L2,R1,2019-06-01,0,No
3,T1188344,174,S1,L1,R4,2019-06-01,0,No
4,T1188345,170,S1,L1,R2,2019-06-01,0,No
...,...,...,...,...,...,...,...,...
22260,T1210601,186,S2,L5,R2,2019-07-31,0,No
22261,T1210602,11,S4,L2,R1,2019-07-31,0,No
22262,T1210603,185,S1,L1,R3,2019-07-31,0,Yes
22263,T1210604,69,S1,L1,R4,2019-07-31,0,No


In [143]:
sub = pd.DataFrame()

In [146]:
sub["ID"] = test["ID"]
sub["Sales"] = result

In [147]:
sub

,ID,Sales
0,T1188341,67248.875000
1,T1188342,40456.449219
2,T1188343,73546.296875
3,T1188344,40456.449219
4,T1188345,40456.449219
...,...,...
22260,T1210601,20947.482422
22261,T1210602,56178.015625
22262,T1210603,43249.371094
22263,T1210604,33266.292969


In [148]:
sub.to_csv("Submission.csv",index=False)